In [1]:
import synapseclient
import synapseutils
from synapseclient import Project, File, Folder
from synapseclient import Schema, Column, Table, Row, RowSet, as_table_columns
import pandas as pd

In [2]:
syn = synapseclient.Synapse()
syn.login()

Welcome, Victor Baham!



### Install `Levenshtein` module, the documentation for which is [here](https://rapidfuzz.github.io/Levenshtein/)

In [ ]:
pip install levenshtein

### Basic idea:

1. Extract `value` column from `table_AD_model`
2. Loop through each cell in `table_AD_backend`:

    a. In cases where the term does not exist in `value`, suggest 3 alternatives based on Levenshtein distance
    
    b. Otherwise, do nothing

### Create DataFrame from AMP-AD Knowledge Portal - All Data `FileView`

In [3]:
schema_AD_backend = syn.get('syn11346063')
results_AD_backend = syn.tableQuery(f"SELECT * FROM {schema_AD_backend.id}")  
table_AD_backend = pd.read_csv(results_AD_backend.filepath)

table_AD_backend

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_2689/736888456.py:3: DtypeWarning: Columns (14,15,18,19,22,23,24,25,28,29,30,31,41,42,43,44) have mixed types. Specify dtype option on import or set low_memory=False.
  table_AD_backend = pd.read_csv(results_AD_backend.filepath)



,ROW_ID,ROW_VERSION,ROW_ETAG,id,name,study,dataType,assay,organ,tissue,...,benefactorId,projectId,modifiedOn,modifiedBy,dataFileHandleId,metaboliteType,chromosome,modelSystemType,libraryPrep,dataFileSizeBytes
0,2426151,3,60e64259-515c-4609-aaa5-7c2d8ba82dc4,syn2426151,chr1.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268379,3420438,75430845,NaN,NaN,NaN,NaN,643156843.0
1,2426152,3,9844ca5a-a422-40ec-8eab-3168769fd309,syn2426152,chr2.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268426,3420438,75430846,NaN,NaN,NaN,NaN,697794692.0
2,2426153,3,220e4abd-75d5-4378-990e-2497e76e113a,syn2426153,chr4.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268473,3420438,75430848,NaN,NaN,NaN,NaN,586390632.0
3,2426154,3,ca15e26b-8672-4b1f-8aae-9b55852186eb,syn2426154,chr8.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268524,3420438,75430847,NaN,NaN,NaN,NaN,449843238.0
4,2426155,3,dffa1f8a-6b1d-4375-bf9f-397a1fc3a9f0,syn2426155,chr9.chop.dosage.gz,"[""ROSMAP""]","[""genomicVariants""]","[""snpArray""]",NaN,NaN,...,syn5550382,syn2580853,1634580268573,3420438,75430849,NaN,NaN,NaN,NaN,359984384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239666,53470895,1,630ce22a-0d54-448f-a0e1-95d2aea54456,syn53470895,UCI_CollaborativeCross_4mo Abeta_SF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242398176,3427990,132908042,NaN,NaN,NaN,NaN,5477.0
239667,53470896,1,2c713a22-71e1-4ee8-9b6f-2ced232e1a83,syn53470896,UCI_CollaborativeCross_12mo Abeta_IF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242399597,3427990,132908044,NaN,NaN,NaN,NaN,5370.0
239668,53470897,1,a29d6dbf-9e06-4e53-9d74-a67c84e9bc21,syn53470897,UCI_CollaborativeCross_12mo Abeta_SF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242401134,3427990,132908046,NaN,NaN,NaN,NaN,5708.0
239669,53470898,1,ff5659a7-5238-40b0-a70f-f54394bddb21,syn53470898,UCI_CollaborativeCross_NfL CTX IF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242410068,3427990,132908048,NaN,NaN,NaN,NaN,8735.0


### Create DataFrame from AD Data Model

In [4]:
schema_AD_model = syn.get('syn53010627')
results_AD_model = syn.tableQuery(f"SELECT * FROM {schema_AD_model.id}")
table_AD_model = pd.read_csv(results_AD_model.filepath)

table_AD_model

,ROW_ID,ROW_VERSION,key,description,columnType,maximumSize,value,valueDescription,source,module
0,2558,33,ageAssessment,Age of clinical or behavioral assessment.,number,NaN,NaN,NaN,sage.annotations-experimentalData.ageAssessmen...,experimentalData
1,2559,33,ageAssessmentUnits,Age of assessment units.,string,NaN,days,"Age measured in days, a period of 24 hours.",sage.annotations-experimentalData.ageAssessmen...,experimentalData
2,2560,33,ageAssessmentUnits,Age of assessment units.,string,NaN,gestational weeks,Gestational age (written with both weeks and d...,sage.annotations-experimentalData.ageAssessmen...,experimentalData
3,2561,33,ageAssessmentUnits,Age of assessment units.,string,NaN,months,Age measured in calendar months (approximately...,sage.annotations-experimentalData.ageAssessmen...,experimentalData
4,2562,33,ageAssessmentUnits,Age of assessment units.,string,NaN,PCW,Post-Conception Weeks,sage.annotations-experimentalData.ageAssessmen...,experimentalData
...,...,...,...,...,...,...,...,...,...,...
1397,3955,33,individual_animal_modelad_metadata_template,individual animal metadata template schema for...,NaN,NaN,NaN,NaN,sysbio.metadataTemplates-modelad.individual.an...,template
1398,3956,33,individual_human_metadata_template,SysBio AD individual human metadata template s...,NaN,NaN,NaN,NaN,sysbio.metadataTemplates-ad.individual.human,template
1399,3957,33,assay,Metadata describing properites of an assay con...,string,NaN,NaN,NaN,Sage Bionetworks,ADKP
1400,3958,33,ATP6V1A_kd,The ATP6V1A knockdown in hIPSC neurons (ATP6V...,string,NaN,NaN,NaN,https://www.synapse.org/#!Synapse:syn52201354,ADKP


### Extract `value` column to list for later similarity scoring

In [5]:
ctrl_vals = table_AD_model['value'].astype(str).tolist()
ctrl_vals

['nan',
 'days',
 'gestational weeks',
 'months',
 'PCW',
 'weeks',
 'years',
 'nan',
 'nan',
 'days',
 'gestational weeks',
 'months',
 'PCW',
 'weeks',
 'years',
 'ANOVA',
 'assessment',
 'batch effect correction',
 'chromatin accessible quantitative trait loci detection',
 'Clustering',
 'comparison',
 'Copy number estimation',
 'correlation',
 'covariate specification',
 'data mining',
 'data normalization',
 'de-novo assembly',
 'differential expression',
 'differential methylation',
 'differential network analysis',
 'DNA methylation imputation',
 'dose response study',
 'Enrichment analysis',
 'EWAS',
 'expression quantitative trait loci detection',
 'Gene expression comparison',
 'Genome-Wide Association',
 'genome-wide pleiotropy',
 'genotypeAncestry',
 'genotype imputation',
 'hidden confounder detection',
 'manifold learning',
 'Mendelian randomization analysis',
 'methylation quantitative trait loci detection',
 'network analysis',
 'network driver detection',
 'outlier det

### Extract columns whose values are of the format `stringList`

In [6]:
sl_cols = table_AD_backend.columns.tolist()
sl_cols = ['study', 'dataType', 'assay','tissue',
            'species', 'sex','grant', 'modelSystemName', 
            'cellType', 'group', 'metaboliteType']

### Function to *clean* columns (remove extra characters)

In [7]:
def clean_cols(df_1):
    for col in sl_cols:          
        df_1[col] = df_1[col].str.replace("]", "")
        df_1[col] = df_1[col].str.replace("[", "")
        df_1[col] = df_1[col].str.replace("\"", "")
    
    return df_1

### Return new DataFrame without brackets and quotation marks

In [8]:
nu_df = clean_cols(table_AD_backend)
nu_df

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_2689/3217499714.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_1[col] = df_1[col].str.replace("]", "")

[WARNING] /var/folders/lg/w3yckgy936lc0r6yw790hjs00000gr/T/ipykernel_2689/3217499714.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_1[col] = df_1[col].str.replace("[", "")



,ROW_ID,ROW_VERSION,ROW_ETAG,id,name,study,dataType,assay,organ,tissue,...,benefactorId,projectId,modifiedOn,modifiedBy,dataFileHandleId,metaboliteType,chromosome,modelSystemType,libraryPrep,dataFileSizeBytes
0,2426151,3,60e64259-515c-4609-aaa5-7c2d8ba82dc4,syn2426151,chr1.chop.dosage.gz,ROSMAP,genomicVariants,snpArray,NaN,NaN,...,syn5550382,syn2580853,1634580268379,3420438,75430845,NaN,NaN,NaN,NaN,643156843.0
1,2426152,3,9844ca5a-a422-40ec-8eab-3168769fd309,syn2426152,chr2.chop.dosage.gz,ROSMAP,genomicVariants,snpArray,NaN,NaN,...,syn5550382,syn2580853,1634580268426,3420438,75430846,NaN,NaN,NaN,NaN,697794692.0
2,2426153,3,220e4abd-75d5-4378-990e-2497e76e113a,syn2426153,chr4.chop.dosage.gz,ROSMAP,genomicVariants,snpArray,NaN,NaN,...,syn5550382,syn2580853,1634580268473,3420438,75430848,NaN,NaN,NaN,NaN,586390632.0
3,2426154,3,ca15e26b-8672-4b1f-8aae-9b55852186eb,syn2426154,chr8.chop.dosage.gz,ROSMAP,genomicVariants,snpArray,NaN,NaN,...,syn5550382,syn2580853,1634580268524,3420438,75430847,NaN,NaN,NaN,NaN,449843238.0
4,2426155,3,dffa1f8a-6b1d-4375-bf9f-397a1fc3a9f0,syn2426155,chr9.chop.dosage.gz,ROSMAP,genomicVariants,snpArray,NaN,NaN,...,syn5550382,syn2580853,1634580268573,3420438,75430849,NaN,NaN,NaN,NaN,359984384.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
239666,53470895,1,630ce22a-0d54-448f-a0e1-95d2aea54456,syn53470895,UCI_CollaborativeCross_4mo Abeta_SF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242398176,3427990,132908042,NaN,NaN,NaN,NaN,5477.0
239667,53470896,1,2c713a22-71e1-4ee8-9b6f-2ced232e1a83,syn53470896,UCI_CollaborativeCross_12mo Abeta_IF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242399597,3427990,132908044,NaN,NaN,NaN,NaN,5370.0
239668,53470897,1,a29d6dbf-9e06-4e53-9d74-a67c84e9bc21,syn53470897,UCI_CollaborativeCross_12mo Abeta_SF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242401134,3427990,132908046,NaN,NaN,NaN,NaN,5708.0
239669,53470898,1,ff5659a7-5238-40b0-a70f-f54394bddb21,syn53470898,UCI_CollaborativeCross_NfL CTX IF.csv,NaN,NaN,NaN,NaN,NaN,...,syn51713893,syn2580853,1707242410068,3427990,132908048,NaN,NaN,NaN,NaN,8735.0


### Import `ratio` module from `Levenshtein`

In [9]:
from Levenshtein import ratio

### For now, let's just try the process on the `assay` column

In [10]:
idx = 0

nu_df['assay'] = nu_df['assay'].astype(str)
for assay in nu_df['assay'].tolist():
    if assay in ctrl_vals:
        pass
    else:
        val_to_sim = {str(val): ratio(assay, val) for val in ctrl_vals}
        nu_vts = list(dict(sorted(val_to_sim.items(), key=lambda x:x[1])[-3:]))
        print("Do you want to change '{}' to any of 0: '{}', 1: '{}', or 2: '{}'? (Enter 0, 1, 2, or 'no'. 'q' to quit)".
              format(assay, nu_vts[0], nu_vts[1], nu_vts[2]))
        choice = input()
        if choice == "no":
            pass
        elif choice == '0':
            nu_df.loc[idx, 'assay'] = nu_vts[0]
            color = 'blue'
        elif choice == '1':
            nu_df.loc[idx, 'assay'] = nu_vts[1]
        elif choice == '2':
            nu_df.loc[idx, 'assay'] = nu_vts[2]
        elif choice == 'q':
            break
        print(nu_df.iloc[idx])
    idx += 1

Do you want to change 'rnaSeq,LC-MSMS' to any of 0: 'HPLC-MSMS', 1: 'UPLC-MSMS', or 2: 'LC-MSMS'? (Enter 0, 1, 2, or 'no'. 'q' to quit)
0
ROW_ID                                                           11336862
ROW_VERSION                                                             1
ROW_ETAG                             d22b9452-bd45-4283-a4ce-0db3b9fde83f
id                                                            syn11336862
name                    DrosophilaTau model study_RNAseqProteomics_fly...
study                                                                 NaN
dataType                                       proteomics, geneExpression
assay                                                           HPLC-MSMS
organ                                                                 NaN
tissue                                                                NaN
species                                           Drosophila melanogaster
sex                                             

### `Ctrl` + `F` within `nu_df` for the changed values

In [13]:
print(nu_df[nu_df['assay'] == 'HPLC-MSMS'])
print(nu_df[nu_df['assay'] == 'UPLC-MSMS'])

         ROW_ID  ROW_VERSION                              ROW_ETAG  \
53459  11336862            1  d22b9452-bd45-4283-a4ce-0db3b9fde83f   

                id                                               name study  \
53459  syn11336862  DrosophilaTau model study_RNAseqProteomics_fly...   NaN   

                         dataType      assay organ tissue  ... benefactorId  \
53459  proteomics, geneExpression  HPLC-MSMS   NaN    NaN  ...  syn17083696   

        projectId     modifiedOn modifiedBy dataFileHandleId metaboliteType  \
53459  syn2580853  1634771323435    3420438         73743001            NaN   

      chromosome modelSystemType libraryPrep dataFileSizeBytes  
53459        NaN          animal         NaN             463.0  

[1 rows x 46 columns]
          ROW_ID  ROW_VERSION                              ROW_ETAG  \
41842    9779079            1  06be6230-daab-4c22-acf2-2b54055b31b5   
41843    9779088            1  f63e9f0c-349c-4704-a926-ca9e058a1fe3   
41844    9779093